In [10]:
!pip install openai tiktoken faiss-cpu

  Using cached openai-0.28.1-py3-none-any.whl (76 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.3 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 7.2 MB/s eta 0:00:00m eta 0:00:010:00:01


In [11]:
import os
import re
import time
import openai
import json
import glob
import pandas as pd 
openai.api_key  = "sk-"
os.environ["OPENAI_API_KEY"] = openai.api_key

In [3]:
def get_completion(messages,
                   model="gpt-3.5-turbo",
                   temperature=0.4):

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )

    content = response.choices[0].message["content"]
    return content

In [4]:
def get_prompt(num_pairs, api, instruct, code, test):
    return """
Act as Python programmer:

Taking into consideration:
 * Variations in the names of input functions and variables will not be considered distinct code variations.
 * implement all needed code in the program.
 * The response will be in JSON format:
    [
    {
    "number":"HERE_GO_NUMBER",
    "instruct": "HERE_GO_INSTRUCT",
    "code": "HERE_GO_CODE",
    "test": ["HERE_GO_TEST_1","HERE_GO_TEST_2","HERE_GO_TEST_3"]
    },
    {
    "number":"HERE_GO_NUMBER",
    "instruct": "HERE_GO_INSTRUCT",
    "code": "HERE_GO_CODE",
    "test": ["HERE_GO_TEST_1","HERE_GO_TEST_2","HERE_GO_TEST_3"]
    },
    ............
    ]


## Task:
=================
You aim to generate """+str(num_pairs)+""" distinct pairs consisting of natural language instructions and fully functional Python program files. Each program should consist of more than 10 lines of code and demonstrate the correct utilization of the Python API \""""+api+"""\" These pairs are intended to evaluate a Python programmer's proficiency in correctly employing various aspects of the \""""+api+"""\" API. Additionally, for each program, specify three unit tests that can be performed to validate the code's functionality.

## Response:
=================
[
{
"number":1,
"instruct": \""""+instruct+"""\",
"code": '''"""+code+"""''',
"test": """+str(test)+"""
},
{
"number":2,
"instruct": "
"""

def get_prompt_one_instant(api, pre_code):
    return """
Act as Python programmer:

Taking into consideration:
 * Variations in the names of input functions and variables will not be considered distinct code variations.
 * implement all needed code in the program.
 * The response will be in Dictionary format:
    {
    "instruct": "HERE_GO_INSTRUCT",
    "code": "HERE_GO_CODE",
    "test": ["HERE_GO_TEST_1","HERE_GO_TEST_2","HERE_GO_TEST_3"]
    }


## Task:
=================
Generate dictionary consisting of natural language instructions and fully functional Python program files. Each program should consist of at least than 10 lines of code and demonstrate the correct utilization of the Python API \""""+api+"""\". These dictionary is intended to evaluate a Python programmer's proficiency in correctly employing various aspects of the \""""+api+"""\" API. Additionally, for each program, specify three unit tests that can be performed to validate the code's functionality.

The dictionary should be totaly distinct from the following ones:
"""+str(pre_code)+"""

## Response:
=================
"""

def get_dict(instruct, code, test):
    return [{
"instruct": "\""+instruct+"\"",
"code": "\""+code+"\"",
"test": "\""+str(test)+"\""
}]

def get_message(num_pairs, api, instruct, code, test):
    system_message = """
Act as Python programmer:

Taking into consideration:
 * Variations in the names of input functions and variables will not be considered distinct code variations.
 * implement all needed code in the program.
 * The response will be in JSON format:
    [
    {
    "number":"HERE_GO_NUMBER",
    "instruct": "HERE_GO_INSTRUCT",
    "code": "HERE_GO_CODE",
    "test": ["HERE_GO_TEST_1","HERE_GO_TEST_2","HERE_GO_TEST_3"]
    },
    {
    "number":"HERE_GO_NUMBER",
    "instruct": "HERE_GO_INSTRUCT",
    "code": "HERE_GO_CODE",
    "test": ["HERE_GO_TEST_1","HERE_GO_TEST_2","HERE_GO_TEST_3"]
    },
    ............
    ]
 """
    user_message = """
## Task:
=================
You aim to generate """+str(num_pairs)+""" distinct pairs consisting of natural language instructions and fully functional Python program files. Each program should consist of more than 10 lines of code and demonstrate the correct utilization of the Python API \""""+api+"""\" These pairs are intended to evaluate a Python programmer's proficiency in correctly employing various aspects of the \""""+api+"""\" API. Additionally, for each program, specify three unit tests that can be performed to validate the code's functionality.

## Response:
=================
[
{
"number":1,
"instruct": \""""+instruct+"""\",
"code": '''"""+code+"""''',
"test": """+str(test)+"""
},
{
"number":2,
"instruct": """
    return [{'role':'system',
 'content':system_message},
{'role':'user',
 'content':user_message}]

In [5]:
# load first shot Data
data = json.load(open('pairs_data/first_shot.json'))

## Prompt For ChatGPT

In [99]:
i = 45
print(data[i]['name'], "................................")
print("..........................................................\n\n")
# load code
with open(data[i]['code'], "r") as f:
    code = f.read()
prompt = get_prompt(41, data[i]['name'], data[i]['instruct'], code, data[i]['test'])
print(prompt)

tensorflow ................................
..........................................................



Act as Python programmer:

Taking into consideration:
 * Variations in the names of input functions and variables will not be considered distinct code variations.
 * implement all needed code in the program.
 * The response will be in JSON format:
    [
    {
    "number":"HERE_GO_NUMBER",
    "instruct": "HERE_GO_INSTRUCT",
    "code": "HERE_GO_CODE",
    "test": ["HERE_GO_TEST_1","HERE_GO_TEST_2","HERE_GO_TEST_3"]
    },
    {
    "number":"HERE_GO_NUMBER",
    "instruct": "HERE_GO_INSTRUCT",
    "code": "HERE_GO_CODE",
    "test": ["HERE_GO_TEST_1","HERE_GO_TEST_2","HERE_GO_TEST_3"]
    },
    ............
    ]


## Task:
You aim to generate 41 distinct pairs consisting of natural language instructions and fully functional Python program files. Each program should consist of more than 10 lines of code and demonstrate the correct utilization of the Python API "tensorflow" These 

## Run in steps on API

In [94]:
i = 5
print(data[i]['name'], "................................")
print("..........................................................\n\n")
# load code
content = ""
# with open('pairs_data/API/gpt-3.5-turbo-16k/'+str(data[i]['name'])+'.txt', "r") as f:
#     content = f.read()
j = 2
while True:
    with open(data[i]['code'], "r") as f:
        code = f.read()
    print("-------------------------"+str(j)+"----------------------------")
    if j == 42:
        break
    try:
        cont = content
        if j != 2:
            if "]" in content[-2:]: cont = content[:-2]
            cont += """,\n{\n"number":"""+str(j)+""",\n"instruct": """
        message = get_message(j, data[i]['name'], data[i]['instruct'], code, data[i]['test'])
        message[1]["content"] += cont
        res = get_completion(message, model="gpt-3.5-turbo-16k", temperature=0)
        content = cont
        content += res
        j+=1
    except Exception as e: 
        print(e)
        print("Sleeping.....................")
        time.sleep(j*50)
        
    with open('pairs_data/API/gpt-3.5-turbo-16k/'+str(data[i]['name'])+'.txt', 'w') as f:
        f.write(content)
        
        
message = get_message(j-1, data[i]['name'], data[i]['instruct'], code, data[i]['test'])
with open('pairs_data/API/gpt-3.5-turbo-16k/'+str(data[i]['name'])+'.txt', 'w') as f:
    f.write(message[1]["content"]+content)

diffusers ................................
..........................................................


-------------------------2----------------------------
-------------------------3----------------------------
-------------------------4----------------------------
-------------------------5----------------------------
-------------------------6----------------------------
-------------------------7----------------------------
-------------------------8----------------------------
-------------------------9----------------------------
-------------------------10----------------------------
-------------------------11----------------------------
-------------------------12----------------------------
-------------------------13----------------------------
-------------------------14----------------------------
-------------------------15----------------------------
-------------------------16----------------------------
-------------------------17----------------------------
--------

In [221]:
paths = glob.glob('pairs_data/gpt_data/*')
i = 0
data = [] 
for path in paths:
    api_name = os.path.basename(path).split(".")[0]
    with open(path) as f:
        content = f.read()
        content = content.split("Response:\n=================\n")[1]
        numbers = re.findall(r"{\n*\"number\n*\":([\d]*?),\n\"instruct\":\n* \"", content)
        instructs = re.findall(r",\n*\"instruct\":\n* \"([\s\S]*?)\",?\n*\"\n*code\":\n* '''", content)
        codes = re.findall(r"\",?\n*\"\n*code\":\n* '''([\s\S]*?)'''?,?\n*\"\n*test\":", content)
        tests = re.findall(r"'''?,?\n*\"\n*test\":\n* ([\s\S]*?)\n?}", content)
        for number, inst, code, test in zip(numbers, instructs, codes, tests):
            i += 1
            tsts = re.findall(r"[\"\']?--\n* ?([\s\S]*?)[\"\']\n*[,\]]", re.sub(r'\s+', ' ', test))
            path_code = "pairs_data/cleaned_data/code/"+api_name
            if not os.path.exists(path_code):
                os.mkdir(path_code)
            with open(path_code+"/"+api_name+"_"+number+".py", 'w') as file:
                file.write(code)
            data.append([i, api_name, int(number), re.sub(r'\s+', ' ', inst), "code/"+api_name+"/"+api_name+"_"+number+".py"]+ list(set(tsts)))

In [223]:
df = pd.DataFrame(data, columns=['#', 'API', 'API_CODE_#','INSTRUCT','CODE_PATH',"TEST_1","TEST_2","TEST_3","TEST_4","TEST_5"]) 

In [224]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2009 entries, 0 to 2008
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           2009 non-null   int64 
 1   API         2009 non-null   object
 2   API_CODE_#  2009 non-null   int64 
 3   INSTRUCT    2009 non-null   object
 4   CODE_PATH   2009 non-null   object
 5   TEST_1      2009 non-null   object
 6   TEST_2      2009 non-null   object
 7   TEST_3      1950 non-null   object
 8   TEST_4      44 non-null     object
 9   TEST_5      9 non-null      object
dtypes: int64(2), object(8)
memory usage: 157.1+ KB


## Remove Duplicates

In [225]:
duplicates = list(df[df['INSTRUCT'].duplicated()]["CODE_PATH"])
for path in duplicates:
    path = "pairs_data/cleaned_data/"+ path
    if os.path.exists(path):
        os.remove(path)

In [226]:
df_no_duplicates = df.drop_duplicates(subset=['INSTRUCT']).reset_index(drop=True)
df_no_duplicates.info(())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1293 entries, 0 to 1292
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           1293 non-null   int64 
 1   API         1293 non-null   object
 2   API_CODE_#  1293 non-null   int64 
 3   INSTRUCT    1293 non-null   object
 4   CODE_PATH   1293 non-null   object
 5   TEST_1      1293 non-null   object
 6   TEST_2      1293 non-null   object
 7   TEST_3      1251 non-null   object
 8   TEST_4      10 non-null     object
 9   TEST_5      4 non-null      object
dtypes: int64(2), object(8)
memory usage: 101.1+ KB


In [227]:
df_no_duplicates.to_csv('pairs_data/cleaned_data/data.csv', index=False)